In [2]:
import json

import os

from data_utils import get_author_ids, YEARS, get_authors_from_follower_lists, DATA_PATH, get_authors_from_keys, \
    write_error_users
from twitter_utils import get_api, get_follower_lists

True

In [16]:
author_ids = set(get_author_ids(2016, save=True))
author_ids.update(get_author_ids(2020, save=True))
author_ids = list(sorted(author_ids))
print(len(author_ids))

In [5]:
api = get_api()

In [29]:
exception_users = dict()

In [ ]:
for year in YEARS:
    out_path = os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl')
    existing_users = get_authors_from_follower_lists(out_path)
    print(f'found {len(existing_users)} scraped users')

    fpath = os.path.join(DATA_PATH, f'polls_{year}_author_ids.json')
    author_ids = get_authors_from_keys(fpath)
    author_ids = list(sorted(id for id in author_ids if id not in existing_users))

    exception_users = get_follower_lists(author_ids, out_path, api)

In [31]:
len(exception_users)

60

In [41]:
fpath = os.path.join(DATA_PATH, 'authors_missing.jsonl')
write_error_users(exception_users, fpath)